In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
def gen_traindata():
    data = np.load("./Lorenz.npz")
    return data["t"], data["y"]

In [3]:
N_inside = 500
N_bound = 10
inside_t = lhs(1,N_inside)*(3-0)
bound_t = np.zeros((N_bound,1))
real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

inside_t = torch.from_numpy(inside_t).float()
bound_t = torch.from_numpy(bound_t).float()
real_bound = torch.from_numpy(real_bound).float()

inside_t.requires_grad_()

###########GPU###########
inside_t = inside_t.cuda()
bound_t = bound_t.cuda()
real_bound = real_bound.cuda()
###########GPU###########

In [4]:
train_t,train_data = gen_traindata()

train_t = torch.from_numpy(train_t).float()
train_data = torch.from_numpy(train_data).float()

n=6
np.random.seed(5678)
index_a = np.random.choice(np.arange(25,dtype=int),n,replace=False)
train_t = train_t[index_a]
train_data = train_data[index_a]

train_t.requires_grad_()

###########GPU###########
train_t = train_t.cuda()
train_data = train_data.cuda()
###########GPU###########

In [5]:
train_t.shape

torch.Size([6, 1])

In [6]:
C1 = torch.tensor(1.0, requires_grad=True)
C2 = torch.tensor(1.0, requires_grad=True)
C3 = torch.tensor(1.0, requires_grad=True)

## PINN

In [7]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs1 = NN_H2(1, 40, 2, 3)
PINNs1.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1,C2,C3], 'lr': 0.001})

nIter1 = 10000

loss_all_1 = []
test_loss_1 = []
C1_a_list = []
C2_a_list = []
C3_a_list = []

loss1_value = 1
it = 0

while  it<20000:
    ##### loss_Bi  ######
    E_bound = PINNs1(bound_t)
    loss_bound = torch.mean(torch.square(E_bound[:,0:1]-real_bound[:,0:1]))+\
                 torch.mean(torch.square(E_bound[:,1:2]-real_bound[:,1:2]))+\
                 torch.mean(torch.square(E_bound[:,2:3]-real_bound[:,2:3]))
    
    ##### loss f  ######    
    E_inside = PINNs1(inside_t)
    E_inside_x = E_inside[:,0:1]
    E_inside_y = E_inside[:,1:2]   
    E_inside_z = E_inside[:,2:3]   
    
    d_x_t = autograd.grad(outputs=E_inside_x, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_y_t = autograd.grad(outputs=E_inside_y, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_z_t = autograd.grad(outputs=E_inside_z, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_z),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]      

    loss_f = torch.mean(torch.square(d_x_t-(C1*(E_inside_y-E_inside_x))))+\
             torch.mean(torch.square(d_y_t-(E_inside_x*(C2-E_inside_z)-E_inside_y)))+\
             torch.mean(torch.square(d_z_t-(E_inside_x*E_inside_y-C3*E_inside_z)))
    
    ##### loss observation  ######        
    E_observation = PINNs1(train_t)
    
    loss_observation = torch.mean(torch.square(E_observation[:,0:1]-train_data[:,0:1]))+\
                       torch.mean(torch.square(E_observation[:,1:2]-train_data[:,1:2]))+\
                       torch.mean(torch.square(E_observation[:,2:3]-train_data[:,2:3]))

    loss = loss_bound+loss_f+2*loss_observation
    
    loss_all_1.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    if (it+1) % 1000 == 0 or it==0:
        print('It:', it, 'Loss:', loss.item())
        
        N_inside = 500
        N_bound = 10
        inside_t = lhs(1,N_inside)*(3-0)
        bound_t = np.zeros((N_bound,1))
        real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

        inside_t = torch.from_numpy(inside_t).float()
        bound_t = torch.from_numpy(bound_t).float()
        real_bound = torch.from_numpy(real_bound).float()

        inside_t.requires_grad_()

        ###########GPU###########
        inside_t = inside_t.cuda()
        bound_t = bound_t.cuda()
        real_bound = real_bound.cuda()
        ###########GPU###########
        
        C1_a_list.append(C1.detach().item())
        C2_a_list.append(C2.detach().item())
        C3_a_list.append(C3.detach().item())
        print('C1:',C1,'C2:',C2,'C3:',C3)
         
    it = it + 1        
loss1_value    

It: 0 Loss: 1325.8997802734375
C1: tensor(0.9990, requires_grad=True) C2: tensor(0.9990, requires_grad=True) C3: tensor(0.9990, requires_grad=True)
It: 999 Loss: 158.54417419433594
C1: tensor(0.2212, requires_grad=True) C2: tensor(1.4431, requires_grad=True) C3: tensor(0.2973, requires_grad=True)
It: 1999 Loss: 133.23974609375
C1: tensor(0.4648, requires_grad=True) C2: tensor(2.6030, requires_grad=True) C3: tensor(0.1605, requires_grad=True)
It: 2999 Loss: 123.96554565429688
C1: tensor(0.6235, requires_grad=True) C2: tensor(3.8289, requires_grad=True) C3: tensor(0.1341, requires_grad=True)
It: 3999 Loss: 117.2294921875
C1: tensor(0.7819, requires_grad=True) C2: tensor(5.0305, requires_grad=True) C3: tensor(0.0995, requires_grad=True)
It: 4999 Loss: 111.18192291259766
C1: tensor(0.9031, requires_grad=True) C2: tensor(6.2086, requires_grad=True) C3: tensor(0.1007, requires_grad=True)
It: 5999 Loss: 105.47624206542969
C1: tensor(1.0358, requires_grad=True) C2: tensor(7.3647, requires_grad

0.012331778183579445

## GA-PINN

In [9]:
N_inside = 500
N_bound = 10
inside_t = lhs(1,N_inside)*(3-0)
bound_t = np.zeros((N_bound,1))
real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

inside_t = torch.from_numpy(inside_t).float()
bound_t = torch.from_numpy(bound_t).float()
real_bound = torch.from_numpy(real_bound).float()

inside_t.requires_grad_()

###########GPU###########
inside_t = inside_t.cuda()
bound_t = bound_t.cuda()
real_bound = real_bound.cuda()
###########GPU###########

In [10]:
C1_b = torch.tensor(1.0, requires_grad=True)
C2_b = torch.tensor(1.0, requires_grad=True)
C3_b = torch.tensor(1.0, requires_grad=True)

In [11]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs2 = NN_H2(1, 40, 2, 3)
PINNs2.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs2.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs2.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1_b,C2_b,C3_b], 'lr': 0.001})

discriminator = get_discriminator(4, 40, 2, 1)
discriminator.cuda()

import torch.nn.init as init
for name, param in discriminator.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)
        
optimizer2 = optim.Adam(discriminator.parameters(), lr=5e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

nIter1 = 10000

loss_all_2 = []
test_loss_2 = []
C1_b_list = []
C2_b_list = []
C3_b_list = []

loss1_value = 1
it = 0

while  it<20000:
    
    if it <=1000:    
    
        ############ loss D ###########
        pre_H = PINNs2(train_t)

        d_fake = discriminator(torch.cat((train_t,pre_H.detach()),1))
        d_real = discriminator(torch.cat((train_t,train_data),1))

        loss_d = torch.mean(1-d_real)+torch.mean(d_fake)

        optimizer2.zero_grad()
        loss_d.backward()
        optimizer2.step()  

        ####### loss G#######
        pre_H = PINNs2(train_t)  

        d_fake = discriminator(torch.cat((train_t,pre_H),1))

        loss_L = torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - train_data))

        optimizer1.zero_grad()
        loss_L.backward()
        optimizer1.step()     
    
    
    ##### loss_Bi  ######
    E_bound = PINNs2(bound_t)
    loss_bound = torch.mean(torch.square(E_bound[:,0:1]-real_bound[:,0:1]))+\
                 torch.mean(torch.square(E_bound[:,1:2]-real_bound[:,1:2]))+\
                 torch.mean(torch.square(E_bound[:,2:3]-real_bound[:,2:3]))
    
    ##### loss f  ######    
    E_inside = PINNs2(inside_t)
    E_inside_x = E_inside[:,0:1]
    E_inside_y = E_inside[:,1:2]   
    E_inside_z = E_inside[:,2:3]   
    
    d_x_t = autograd.grad(outputs=E_inside_x, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_y_t = autograd.grad(outputs=E_inside_y, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    
    d_z_t = autograd.grad(outputs=E_inside_z, inputs=inside_t,
                              grad_outputs=torch.ones_like(E_inside_z),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]      

    loss_f = torch.mean(torch.square(d_x_t-(C1_b*(E_inside_y-E_inside_x))))+\
             torch.mean(torch.square(d_y_t-(E_inside_x*(C2_b-E_inside_z)-E_inside_y)))+\
             torch.mean(torch.square(d_z_t-(E_inside_x*E_inside_y-C3_b*E_inside_z)))
    
    ##### loss observation  ######        
    E_observation = PINNs2(train_t)
    
    loss_observation = torch.mean(torch.square(E_observation[:,0:1]-train_data[:,0:1]))+\
                       torch.mean(torch.square(E_observation[:,1:2]-train_data[:,1:2]))+\
                       torch.mean(torch.square(E_observation[:,2:3]-train_data[:,2:3]))

    loss = loss_bound+loss_f+2*loss_observation
    
    loss_all_2.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    
    if (it+1) % 1000 == 0 or it==0:
        print('It:', it, 'Loss:', loss.item())
        
        N_inside = 500
        N_bound = 10
        inside_t = lhs(1,N_inside)*(3-0)
        bound_t = np.zeros((N_bound,1))
        real_bound = np.concatenate((-8*np.ones((N_bound,1)),7*np.ones((N_bound,1)),27*np.ones((N_bound,1))),axis=1)

        inside_t = torch.from_numpy(inside_t).float()
        bound_t = torch.from_numpy(bound_t).float()
        real_bound = torch.from_numpy(real_bound).float()

        inside_t.requires_grad_()

        ###########GPU###########
        inside_t = inside_t.cuda()
        bound_t = bound_t.cuda()
        real_bound = real_bound.cuda()
        ###########GPU###########
        
        C1_b_list.append(C1_b.detach().item())
        C2_b_list.append(C2_b.detach().item())
        C3_b_list.append(C3_b.detach().item())
        print('C1:',C1_b,'C2:',C2_b,'C3:',C3_b)
        
    it = it + 1        
loss1_value    

It: 0 Loss: 1305.174560546875
C1: tensor(0.9990, requires_grad=True) C2: tensor(0.9990, requires_grad=True) C3: tensor(0.9990, requires_grad=True)
It: 999 Loss: 142.97412109375
C1: tensor(0.3297, requires_grad=True) C2: tensor(1.7282, requires_grad=True) C3: tensor(0.2111, requires_grad=True)
It: 1999 Loss: 127.29408264160156
C1: tensor(0.6482, requires_grad=True) C2: tensor(3.0684, requires_grad=True) C3: tensor(0.1186, requires_grad=True)
It: 2999 Loss: 120.51895141601562
C1: tensor(0.7657, requires_grad=True) C2: tensor(4.3173, requires_grad=True) C3: tensor(0.1070, requires_grad=True)
It: 3999 Loss: 113.29388427734375
C1: tensor(0.8922, requires_grad=True) C2: tensor(5.5035, requires_grad=True) C3: tensor(0.0990, requires_grad=True)
It: 4999 Loss: 108.39942932128906
C1: tensor(0.9877, requires_grad=True) C2: tensor(6.6664, requires_grad=True) C3: tensor(0.1002, requires_grad=True)
It: 5999 Loss: 103.38323974609375
C1: tensor(1.0733, requires_grad=True) C2: tensor(7.8072, requires_g

0.008427070453763008

## Results

In [ ]:
### Save data

In [ ]:
# np.save('../experimental_data/J3/C1_a_list',C1_a_list)
# np.save('../experimental_data/J3/C1_b_list',C1_b_list)
# np.save('../experimental_data/J3/C2_a_list',C2_a_list)
# np.save('../experimental_data/J3/C2_b_list',C2_b_list)
# np.save('../experimental_data/J3/C3_a_list',C3_a_list)
# np.save('../experimental_data/J3/C3_b_list',C3_b_list)